In [90]:
import pandas as pd
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from matplotlib import pyplot as plt
from sklearn.metrics import plot_precision_recall_curve
import numpy as np
from sklearn.model_selection import GridSearchCV

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [91]:
pip install openpyxl

In [92]:
df = pd.read_excel('text_rating_final.xlsx')

In [93]:
df.shape

(32436, 3)

In [94]:
df.columns=["comment", "type", 2]

In [95]:
df_new = df.drop([2], axis=1)

In [96]:
df_new.head(9)

,comment,type
0,"Когда возникнут трудности, они тебе не помогут...",0
1,Кривая национализация это политический компром...,-1
2,Такой вид биологического оружия не действует н...,-2
3,В Эль-Кусейре /к западу от Хомса/ сирийские по...,0
4,"В том числе, ограничениями на продажу спиртных...",0
5,"бактериофобия, верминофобия, вермифобия, гельм...",-1
6,Авиа и артиллерийские удары были нанесены по у...,0
7,Президент Сирии также ответил на обвинения в а...,-1
8,Во двор въехали две лошади. Это были сыновья Т...,0


In [97]:
df_new.loc[(df_new.type == - 2), 'type'] = -1

In [98]:
df_new.loc[(df_new.type == 2), 'type'] = 1

In [99]:
df_new

,comment,type
0,"Когда возникнут трудности, они тебе не помогут...",0
1,Кривая национализация это политический компром...,-1
2,Такой вид биологического оружия не действует н...,-1
3,В Эль-Кусейре /к западу от Хомса/ сирийские по...,0
4,"В том числе, ограничениями на продажу спиртных...",0
...,...,...
32431,Это помогло Соединенным Штатам прорубить окно ...,0
32432,Она уже вернулась на УИК. Член ПСГ Строгин от ...,-1
32433,Всего было две линии обороны: в первую входили...,-1
32434,5. Западный образ жизни несовместим с выживани...,-1


In [100]:
df_new["type"].value_counts()

0               16967
-1              12264
1                3188
 совершенный        4
Name: type, dtype: int64

In [101]:
pd.isnull(df_new["type"]).value_counts()

False    32423
True        13
Name: type, dtype: int64

In [102]:
df_new = df_new.dropna(subset=['type'])

In [103]:
pd.isnull(df_new["type"]).value_counts()

False    32423
Name: type, dtype: int64

In [104]:
train_df, test_df = train_test_split(df_new, test_size=31000)

In [105]:
pd.isnull(test_df["type"]).value_counts()

False    31000
Name: type, dtype: int64

In [106]:
pd.isnull(train_df["type"]).value_counts()

False    1423
Name: type, dtype: int64

In [107]:
sentence_example = df.iloc[1]["comment"]
tokens = word_tokenize(sentence_example, language="russian")
tokens_without_punctuation = [i for i in tokens if i not in string.punctuation]
russian_stop_words = stopwords.words("russian")
tokens_without_stop_words_and_punctuation = [i for i in tokens_without_punctuation if i not in russian_stop_words]
snowball = SnowballStemmer(language="russian")
stemmed_tokens = [snowball.stem(i) for i in tokens_without_stop_words_and_punctuation]

In [108]:
print(f"Исходный текст: {sentence_example}")
print("-----------------")
print(f"Токены: {tokens}")
print("-----------------")
print(f"Токены без пунктуации: {tokens_without_punctuation}")
print("-----------------")
print(f"Токены без пунктуации и стоп слов: {tokens_without_stop_words_and_punctuation}")
print("-----------------")
print(f"Токены после стемминга: {stemmed_tokens}")
print("-----------------")

Исходный текст: Кривая национализация это политический компромисс. С одной стороны, многочисленные рантье, возникшие вследствие приватизации, сохранили возможность и дальше получать свою ренту. Этот паразитический класс даже существенно увеличился в размере, пополнившись многочисленными представителями силовой бюрократии, не успевшими к первой раздаче. С другой стороны, все они превратились в условных держателей активов, распоряжающихся ими с разрешения правительства, которое накладывает на них разнообразные обременения как социального, так и коррупционного характера.Эта уродливая система, основанная на слегка задрапированном голом насилии, не решая ни одной из проблем, порожденных приватизацией, добавила к ним новые проблемы, ставшие следствием порождаемого ею правового беспредела. Именно попытка осуществить скрытую национализацию привела к окончательному превращению России в мафиозное государство. Путин лечил Россию, но не вылечил.
-----------------
Токены: ['Кривая', 'национализация

In [109]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [110]:
tokenize_sentence(sentence_example)

['крив',
 'национализац',
 'эт',
 'политическ',
 'компромисс',
 'с',
 'одн',
 'сторон',
 'многочислен',
 'рант',
 'возникш',
 'вследств',
 'приватизац',
 'сохран',
 'возможн',
 'дальш',
 'получа',
 'рент',
 'этот',
 'паразитическ',
 'класс',
 'существен',
 'увелич',
 'размер',
 'пополн',
 'многочислен',
 'представител',
 'силов',
 'бюрократ',
 'успевш',
 'перв',
 'раздач',
 'с',
 'сторон',
 'преврат',
 'условн',
 'держател',
 'актив',
 'распоряжа',
 'им',
 'разрешен',
 'правительств',
 'котор',
 'накладыва',
 'разнообразн',
 'обременен',
 'социальн',
 'коррупцион',
 'характера.эт',
 'уродлив',
 'систем',
 'основа',
 'слегк',
 'задрапирова',
 'гол',
 'насил',
 'реш',
 'одн',
 'пробл',
 'порожден',
 'приватизац',
 'добав',
 'нов',
 'проблем',
 'ставш',
 'следств',
 'порожда',
 'е',
 'правов',
 'беспредел',
 'имен',
 'попытк',
 'осуществ',
 'скрыт',
 'национализац',
 'привел',
 'окончательн',
 'превращен',
 'росс',
 'мафиозн',
 'государств',
 'путин',
 'леч',
 'росс',
 'вылеч']

In [111]:
vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))

In [112]:
features = vectorizer.fit_transform(train_df["comment"].apply(lambda features: np.str_(features)))

In [113]:
pd.isnull(train_df["type"]).value_counts()

False    1423
Name: type, dtype: int64

In [114]:
model = LogisticRegression(random_state=0)
model.fit(features, train_df["type"])
# model.fit(features, train_df["type"])

ValueError: Unknown label type: 'unknown'